In [33]:
import time
import numpy as np
import copy

def initialize_domains(board):
    """Initialize domains for each cell: {1-9} for empty cells, {value} for filled cells."""
    domains = {}
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                domains[(i, j)] = set(range(1, 10))
            else:
                domains[(i, j)] = {board[i][j]}
    return domains

def get_related_cells(i, j):
    """Get all cells in the same row, column, and 3x3 box as (i, j), excluding (i, j)."""
    related = set()
   
    for col in range(9):
        if col != j:
            related.add((i, col))

    for row in range(9):
        if row != i:
            related.add((row, j))
    # 3x3 box
    start_row, start_col = 3 * (i // 3), 3 * (j // 3)
    for row in range(start_row, start_row + 3):
        for col in range(start_col, start_col + 3):
            if (row, col) != (i, j):
                related.add((row, col))
    return related

def forward_checking(board, domains, row, col, num):
   
    affected = []
    for r, c in get_related_cells(row, col):
        if board[r][c] == 0 and num in domains[(r, c)]:
            domains[(r, c)].remove(num)
            affected.append((r, c, num))
            if not domains[(r, c)]:
                return False, affected
    return True, affected

def restore_domains(domains, affected):
    """Restore domains for backtracking."""
    for r, c, num in affected:
        domains[(r, c)].add(num)

def solve_sudoku_fc(board, domains):
    # Find an empty cell
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                for num in domains[(i, j)].copy():
                    board[i][j] = num
                    success, affected = forward_checking(board, domains, i, j, num)
                    if success:
                        if solve_sudoku_fc(board, domains):
                            return True
                    # Backtrack
                    board[i][j] = 0
                    restore_domains(domains, affected)
                return False
    return True

# Sample Sudoku puzzle
sudoku_grid = np.array([
    [5,3,0,0,7,0,0,0,0],
    [6,0,0,1,9,5,0,0,0],
    [0,9,8,0,0,0,0,6,0],
    [8,0,0,0,6,0,0,0,3],
    [4,0,0,8,0,3,0,0,1],
    [7,0,0,0,2,0,0,0,6],
    [0,6,0,0,0,0,2,8,0],
    [0,0,0,4,1,9,0,0,5],
    [0,0,0,0,8,0,0,7,9]
])

sudoku_grid1 = np.array([
    [4, 0, 0, 0, 0, 0, 8, 0, 5],
    [0, 3, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 6, 0],
    [0, 0, 0, 0, 8, 0, 4, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 3, 0, 7, 0],
    [5, 0, 0, 2, 0, 0, 0, 0, 0],
    [1, 0, 4, 0, 0, 0, 0, 0, 0]
])

# Measure time for 10 runs
times_fc = []
for _ in range(10):
    board = sudoku_grid1.copy()
    domains = initialize_domains(board)
    start_time = time.time()
    solve_sudoku_fc(board, domains)
    end_time = time.time()
    times_fc.append(end_time - start_time)

avg_time_fc = sum(times_fc) / len(times_fc)
print(f"Average time for Forward Checking (10 runs): {avg_time_fc:.4f} seconds")

Average time for Forward Checking (10 runs): 0.0049 seconds


In [ ]:
import time
import numpy as np
from collections import deque

def initialize_domains(board):
    """Initialize domains for each cell."""
    domains = {}
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                domains[(i, j)] = set(range(1, 10))
            else:
                domains[(i, j)] = {board[i][j]}
    return domains

def get_arcs():

    arcs = []
    
    for i in range(9):
        for j1 in range(9):
            for j2 in range(j1 + 1, 9):
                arcs.append(((i, j1), (i, j2)))
                arcs.append(((i, j2), (i, j1)))
   
    for j in range(9):
        for i1 in range(9):
            for i2 in range(i1 + 1, 9):
                arcs.append(((i1, j), (i2, j)))
                arcs.append(((i2, j), (i1, j)))
    # Box constraints
    for box_row in range(0, 9, 3):
        for box_col in range(0, 9, 3):
            cells = [(box_row + i, box_col + j) for i in range(3) for j in range(3)]
            for idx1, cell1 in enumerate(cells):
                for cell2 in cells[idx1 + 1:]:
                    arcs.append((cell1, cell2))
                    arcs.append((cell2, cell1))
    return arcs

def revise(domains, xi, xj):

    revised = False
    values_to_remove = set()
    for x in domains[xi]:
        if not any(x != y for y in domains[xj]):  # No value in xj supports x
            values_to_remove.add(x)
            revised = True
    domains[xi] -= values_to_remove
    return revised

def ac3(domains, arcs):
    """
    Enforce arc consistency using AC-3 algorithm.
    Return False if any domain becomes empty, else True.
    """
    queue = deque(arcs)
    while queue:
        xi, xj = queue.popleft()
        if revise(domains, xi, xj):
            if not domains[xi]:
                return False
            # Add all arcs (xk, xi) where xk != xj
            for xk, _ in arcs:
                if xk != xj and (xk, xi) in arcs:
                    queue.append((xk, xi))
    return True

def forward_checking(board, domains, row, col, num, arcs):
    
    affected = []
    board[row][col] = num
    domains[(row, col)] = {num}
    # Remove num from related cells' domains
    for r, c in get_related_cells(row, col):
        if board[r][c] == 0 and num in domains[(r, c)]:
            domains[(r, c)].remove(num)
            affected.append((r, c, num))
    # Run AC-3 to propagate
    if not ac3(domains, arcs):
        return False, affected
    return True, affected

def restore_domains(domains, affected, row, col):
    """Restore domains and board for backtracking."""
    for r, c, num in affected:
        domains[(r, c)].add(num)
    domains[(row, col)] = set(range(1, 10))

def solve_sudoku_ac3(board, domains, arcs):
    # Find an empty cell
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                for num in domains[(i, j)].copy():
                    success, affected = forward_checking(board, domains, i, j, num, arcs)
                    if success:
                        if solve_sudoku_ac3(board, domains, arcs):
                            return True
                    # Backtrack
                    board[i][j] = 0
                    restore_domains(domains, affected, i, j)
                return False
    return True

# Sample Sudoku puzzle
sudoku_grid = np.array([
    [5,3,0,0,7,0,0,0,0],
    [6,0,0,1,9,5,0,0,0],
    [0,9,8,0,0,0,0,6,0],
    [8,0,0,0,6,0,0,0,3],
    [4,0,0,8,0,3,0,0,1],
    [7,0,0,0,2,0,0,0,6],
    [0,6,0,0,0,0,2,8,0],
    [0,0,0,4,1,9,0,0,5],
    [0,0,0,0,8,0,0,7,9]
])


arcs = get_arcs()


times_ac3 = []
for _ in range(10):
    board = sudoku_grid.copy()
    domains = initialize_domains(board)
    # Initial AC-3 to reduce domains
    if not ac3(domains, arcs):
        print("Initial AC-3 failed")
        continue
    start_time = time.time()
    solve_sudoku_ac3(board, domains, arcs)
    end_time = time.time()
    times_ac3.append(end_time - start_time)

avg_time_ac3 = sum(times_ac3) / len(times_ac3)
print(f"Average time for Arc Consistency (AC-3) (10 runs): {avg_time_ac3:.4f} seconds")

Average time for Arc Consistency (AC-3) (10 runs): 0.3265 seconds


In [5]:
import time
import copy

def print_board(board):
    for row in board:
        print(" ".join(map(str, row)))

def find_empty(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i, j)
    return None

def is_valid(board, num, pos):
    row, col = pos
    
    # Check row
    if num in board[row]:
        return False
    
    # Check column
    if num in [board[i][col] for i in range(9)]:
        return False
    
    # Check 3x3 box
    box_x = col // 3
    box_y = row // 3
    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x*3, box_x*3 + 3):
            if board[i][j] == num and (i, j) != pos:
                return False
    return True

def solve_sudoku(board):
    empty = find_empty(board)
    if not empty:
        return True
    row, col = empty
    
    for num in range(1, 10):
        if is_valid(board, num, (row, col)):
            board[row][col] = num
            if solve_sudoku(board):
                return True
            board[row][col] = 0
    return False

# Test board (World's Hardest Sudoku)
initial_board = [
    [8,0,0,0,0,0,0,0,0],
    [0,0,3,6,0,0,0,0,0],
    [0,7,0,0,9,0,2,0,0],
    [0,5,0,0,0,7,0,0,0],
    [0,0,0,0,4,5,7,0,0],
    [0,0,0,1,0,0,0,3,0],
    [0,0,1,0,0,0,0,6,8],
    [0,0,8,5,0,0,0,1,0],
    [0,9,0,0,0,0,4,0,0]
]

# Time 10 runs
total_time = 0
for _ in range(10):
    test_board = copy.deepcopy(initial_board)
    start = time.perf_counter()
    solve_sudoku(test_board)
    total_time += time.perf_counter() - start

print(f" {total_time/10:.4f} seconds")
print("\nSolved board:")
print_board(test_board)


 0.4180 seconds

Solved board:
8 1 2 7 5 3 6 4 9
9 4 3 6 8 2 1 7 5
6 7 5 4 9 1 2 8 3
1 5 4 2 3 7 8 9 6
3 6 9 8 4 5 7 2 1
2 8 7 1 6 9 5 3 4
5 2 1 9 7 4 3 6 8
4 3 8 5 2 6 9 1 7
7 9 6 3 1 8 4 5 2


In [4]:
import time
import random
from copy import deepcopy
from collections import defaultdict
import numpy as np

# Define a class to represent the Sudoku CSP
class SudokuCSP:
    def __init__(self, board):
        self.board = board
        self.variables = [(i, j) for i in range(9) for j in range(9)]  # 81 squares
        self.domains = self.initialize_domains()
        self.neighbors = self.initialize_neighbors()
        self.constraints = self.initialize_constraints()

    def initialize_domains(self):
        domains = {}
        for var in self.variables:
            i, j = var
            if self.board[i][j] != 0:
                domains[var] = [self.board[i][j]]  # Fixed value
            else:
                domains[var] = list(range(1, 10))  # Possible values 1-9
        return domains

    def initialize_neighbors(self):
        neighbors = defaultdict(list)
        for i in range(9):
            for j in range(9):
                var = (i, j)
                # Row neighbors
                for col in range(9):
                    if col != j:
                        neighbors[var].append((i, col))
                # Column neighbors
                for row in range(9):
                    if row != i:
                        neighbors[var].append((row, j))
                # Box neighbors
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for r in range(box_row, box_row + 3):
                    for c in range(box_col, box_col + 3):
                        if (r, c) != (i, j):
                            neighbors[var].append((r, c))
        return neighbors

    def initialize_constraints(self):
        constraints = []
        for var in self.variables:
            for neighbor in self.neighbors[var]:
                constraints.append((var, neighbor))
        return constraints

# Function to check if an assignment is consistent
def consistent(board, var, value):
    i, j = var
    # Check row
    for col in range(9):
        if col != j and board[i][col] == value:
            return False
    # Check column
    for row in range(9):
        if row != i and board[row][j] == value:
            return False
    # Check 3x3 box
    box_row, box_col = 3 * (i // 3), 3 * (j // 3)
    for r in range(box_row, box_row + 3):
        for c in range(box_col, box_col + 3):
            if (r, c) != (i, j) and board[r][c] == value:
                return False
    return True

# Basic backtracking search
def backtrack(csp, heuristics=False):
    board = deepcopy(csp.board)
    def backtrack_recursive(assignment):
        if all(board[i][j] != 0 for i, j in csp.variables):
            return board
        var = select_unassigned_variable(csp, board, heuristics)
        for value in order_domain_values(csp, var, board, heuristics):
            if consistent(board, var, value):
                i, j = var
                board[i][j] = value
                result = backtrack_recursive(assignment)
                if result is not None:
                    return result
                board[i][j] = 0
        return None
    return backtrack_recursive({})

# Backtracking with forward checking
def backtrack_fc(csp, heuristics=False):
    board = deepcopy(csp.board)
    domains = deepcopy(csp.domains)
    def forward_check(var, value):
        i, j = var
        removed = []
        for neighbor in csp.neighbors[var]:
            if board[neighbor[0]][neighbor[1]] == 0:
                if value in domains[neighbor]:
                    domains[neighbor].remove(value)
                    removed.append((neighbor, value))
                    if not domains[neighbor]:
                        return None
        return removed
    
    def backtrack_recursive():
        if all(board[i][j] != 0 for i, j in csp.variables):
            return board
        var = select_unassigned_variable(csp, board, heuristics)
        for value in order_domain_values(csp, var, board, heuristics):
            if consistent(board, var, value):
                i, j = var
                board[i][j] = value
                saved = forward_check(var, value)
                if saved is not None:
                    result = backtrack_recursive()
                    if result is not None:
                        return result
                board[i][j] = 0
                if saved:
                    for neighbor, val in saved:
                        domains[neighbor].append(val)
        return None
    return backtrack_recursive()

# AC-3 algorithm for arc consistency
def ac3(csp, arcs=None):
    domains = deepcopy(csp.domains)
    queue = arcs if arcs else csp.constraints[:]
    while queue:
        (xi, xj) = queue.pop(0)
        if revise(csp, xi, xj, domains):
            if not domains[xi]:
                return None
            for xk in csp.neighbors[xi]:
                if xk != xj:
                    queue.append((xk, xi))
    return domains

def revise(csp, xi, xj, domains):
    revised = False
    values = domains[xi][:]
    for x in values:
        if not any(csp.board[xj[0]][xj[1]] == 0 or csp.board[xj[0]][xj[1]] != x for y in domains[xj]):
            domains[xi].remove(x)
            revised = True
    return revised

# Backtracking with arc consistency
def backtrack_ac3(csp, heuristics=False):
    board = deepcopy(csp.board)
    initial_domains = ac3(csp)
    if not initial_domains:
        return None
    def backtrack_recursive(domains):
        if all(board[i][j] != 0 for i, j in csp.variables):
            return board
        var = select_unassigned_variable(csp, board, heuristics, domains)
        for value in order_domain_values(csp, var, board, heuristics):
            if consistent(board, var, value):
                i, j = var
                board[i][j] = value
                saved_domains = deepcopy(domains)
                arcs = [(neighbor, var) for neighbor in csp.neighbors[var] if board[neighbor[0]][neighbor[1]] == 0]
                domains = ac3(csp, arcs)
                if domains:
                    result = backtrack_recursive(domains)
                    if result is not None:
                        return result
                board[i][j] = 0
                domains = saved_domains
        return None
    return backtrack_recursive(initial_domains)

# Variable selection with MRV and Degree heuristic
def select_unassigned_variable(csp, board, heuristics=False, domains=None):
    unassigned = [(i, j) for i, j in csp.variables if board[i][j] == 0]
    if not heuristics:
        return unassigned[0] if unassigned else None
    # MRV
    domains = domains or csp.domains
    mrv = min(unassigned, key=lambda var: len(domains[var]))
    # Degree heuristic (used as tie-breaker)
    degree = max([(var, sum(1 for n in csp.neighbors[var] if board[n[0]][n[1]] == 0)) for var in unassigned if len(domains[var]) == len(domains[mrv])], key=lambda x: x[1])[0]
    return degree

# Value ordering with LCV
def order_domain_values(csp, var, board, heuristics=False):
    domain = [v for v in range(1, 10) if consistent(board, var, v)]
    if not heuristics:
        return domain
    # LCV: sort values by how few options they rule out
    def count_constraints(value):
        count = 0
        for neighbor in csp.neighbors[var]:
            if board[neighbor[0]][neighbor[1]] == 0:
                if value in csp.domains[neighbor]:
                    count += 1
        return count
    return sorted(domain, key=count_constraints)

# Print Sudoku board
def print_board(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            print(board[i][j] if board[i][j] != 0 else ".", end=" ")
        print()
    print()

# Sample Sudoku puzzles (10 different configurations)
puzzles = [
    [
        [5,3,0,0,7,0,0,0,0],
        [6,0,0,1,9,5,0,0,0],
        [0,9,8,0,0,0,0,6,0],
        [8,0,0,0,6,0,0,0,3],
        [4,0,0,8,0,3,0,0,1],
        [7,0,0,0,2,0,0,0,6],
        [0,6,0,0,0,0,2,8,0],
        [0,0,0,4,1,9,0,0,5],
        [0,0,0,0,8,0,0,7,9]
    ],
    # Add 9 more puzzles
    [
        [0,0,5,3,0,0,0,0,0],
        [8,0,0,0,0,0,0,2,0],
        [0,7,0,0,1,0,5,0,0],
        [4,0,0,0,0,5,3,0,0],
        [0,1,0,0,7,0,0,0,6],
        [0,0,3,2,0,0,0,8,0],
        [0,6,0,5,0,0,0,0,9],
        [0,0,4,0,0,0,0,3,0],
        [0,0,0,0,0,9,7,0,0]
    ],
    [
        [7,0,0,0,0,0,2,0,0],
        [0,6,0,0,5,0,0,8,0],
        [0,0,9,7,0,0,0,0,4],
        [0,0,3,9,0,0,0,0,6],
        [0,8,0,0,4,0,0,1,0],
        [4,0,0,0,0,2,7,0,0],
        [8,0,0,0,0,9,3,0,0],
        [0,5,0,0,7,0,0,4,0],
        [0,0,1,0,0,0,0,0,5]
    ],
    [
        [0,0,0,6,0,0,0,0,8],
        [0,8,0,0,9,0,0,3,0],
        [2,0,0,0,0,5,4,0,0],
        [0,0,7,0,3,0,0,0,9],
        [0,6,0,4,0,7,0,1,0],
        [1,0,0,0,5,0,7,0,0],
        [0,0,4,3,0,0,0,0,5],
        [0,9,0,0,7,0,0,6,0],
        [8,0,0,0,0,6,0,0,0]
    ],
    [
        [0,2,0,0,0,8,0,0,0],
        [0,0,7,0,9,0,5,0,0],
        [5,0,0,4,0,0,0,6,0],
        [0,0,8,7,0,0,0,0,2],
        [0,6,0,0,3,0,0,9,0],
        [9,0,0,0,0,1,4,0,0],
        [0,4,0,0,0,9,0,0,7],
        [0,0,2,0,6,0,9,0,0],
        [0,0,0,5,0,0,0,8,0]
    ],
    [
        [4,0,0,0,0,5,0,0,0],
        [0,7,0,0,8,0,0,6,0],
        [0,0,2,9,0,0,8,0,0],
        [0,0,9,4,0,0,0,0,7],
        [0,5,0,0,6,0,0,3,0],
        [8,0,0,0,0,2,5,0,0],
        [0,0,7,0,0,9,1,0,0],
        [0,6,0,0,4,0,0,8,0],
        [0,0,0,5,0,0,0,0,2]
    ],
    [
        [0,0,8,0,0,4,0,0,0],
        [7,0,0,0,6,0,0,0,9],
        [0,5,0,8,0,0,3,0,0],
        [0,0,2,0,0,9,0,0,5],
        [0,7,0,0,8,0,0,4,0],
        [1,0,0,5,0,0,6,0,0],
        [0,0,4,0,0,3,0,9,0],
        [9,0,0,0,5,0,0,0,6],
        [0,0,0,7,0,0,8,0,0]
    ],
    [
        [0,0,0,0,0,7,9,0,0],
        [0,4,0,0,2,0,0,7,0],
        [8,0,0,6,0,0,0,0,3],
        [0,0,6,0,0,8,0,0,9],
        [0,9,0,0,5,0,0,8,0],
        [7,0,0,3,0,0,2,0,0],
        [2,0,0,0,0,9,0,0,6],
        [0,7,0,0,8,0,0,4,0],
        [0,0,5,4,0,0,0,0,0]
    ],
    [
        [0,8,0,0,0,0,7,0,0],
        [0,0,4,7,0,0,0,0,6],
        [7,0,0,0,9,0,0,5,0],
        [0,0,9,0,0,2,0,0,8],
        [0,6,0,0,4,0,0,3,0],
        [2,0,0,5,0,0,9,0,0],
        [0,7,0,0,3,0,0,0,4],
        [8,0,0,0,0,6,5,0,0],
        [0,0,2,0,0,0,0,9,0]
    ],
    [
        [0,0,0,8,0,0,0,0,4],
        [0,7,0,0,5,0,0,8,0],
        [9,0,0,0,0,6,7,0,0],
        [0,0,8,0,0,9,0,0,6],
        [0,4,0,0,3,0,0,9,0],
        [7,0,0,4,0,0,2,0,0],
        [0,0,7,9,0,0,0,0,8],
        [0,8,0,0,6,0,0,7,0],
        [6,0,0,0,0,8,0,0,0]
    ]
]

# Timing analysis
def run_experiments():
    methods = [
        ("Plain Backtracking", backtrack, False),
        ("Backtracking + Heuristics", backtrack, True),
        ("Forward Checking", backtrack_fc, False),
        ("Forward Checking + Heuristics", backtrack_fc, True),
        ("Arc Consistency", backtrack_ac3, False),
        ("Arc Consistency + Heuristics", backtrack_ac3, True),
    ]
    avg_times = {name: [] for name, _, _ in methods}
    
    for i, puzzle in enumerate(puzzles):
        print(f"\nRunning puzzle {i+1}")
        csp = SudokuCSP(puzzle)
        for name, method, heuristics in methods:
            start_time = time.time()
            result = method(csp, heuristics)
            elapsed_time = time.time() - start_time
            avg_times[name].append(elapsed_time)
            if i == 0 and result is not None:  # Print result for first puzzle
                print(f"\nSolution for {name}:")
                print_board(result)
    
    # Compute and print average times
    print("\nAverage times over 10 runs (seconds):")
    for name in avg_times:
        avg_time = sum(avg_times[name]) / len(avg_times[name])
        print(f"{name}: {avg_time:.4f}")

# Run experiments
run_experiments()


Running puzzle 1

Solution for Plain Backtracking:
5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
- - - - - - - - - - - -
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
- - - - - - - - - - - -
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 


Solution for Backtracking + Heuristics:
5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
- - - - - - - - - - - -
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
- - - - - - - - - - - -
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 


Solution for Forward Checking:
5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
- - - - - - - - - - - -
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
- - - - - - - - - - - -
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 



KeyboardInterrupt: 